# Building a movie retrieval system

In this tutorial, we're going to build an end-to-end retrieval system for movies. 

A retrieval system is normally the first stage in a multi-stage recommender system and is responsible for retrieving a set of candidates out of a large corpus in response to a user query.

Retrieval models are often composed of two sub-models:

1. A query model computing the query representation (normally a fixed-dimensionality embedding vector) using query features.
2. A candidate model computing the candidate representation (an equally-sized vector) using the candidate features

The outputs of the two models are then multiplied together to give a query-candidate affinity score, with higher scores expressing a better match between the candidate and the query.

In this tutorial, we're going to build and train such a two-tower model using the Movielens dataset.

We're going to:

1. Get our data and split it into a training and test set.
2. Implement a retrieval model.
3. Fit and evaluate it.
4. Export it for efficient serving by building an approximate nearest neighbours (ANN) index.

## The dataset

The Movielens dataset is a classic dataset from the [GroupLens](https://grouplens.org/datasets/movielens/) research group at the University of Minnesota. It contains a set of ratings given to movies by a set of users, and is a workhorse of recommender system research.

The data can be treated in two ways:

1. It can be interpreted as expressesing which movies the users watched (and rated), and which they did not. This is a form of implicit feedback, where users' watches tell us which things they prefer to see and which they'd rather not see.
2. It can also be seen as expressesing how much the users liked the movies they did watch. This is a form of explicit feedback: given that a user watched a movie, we can tell roughly how much they liked by looking at the rating they have given.

In this tutorial, we are focusing on a retrieval system: a model that predicts a set of movies from the catalogue that the user is likely to watch. Often, implicit data is more useful here, and so we are going to treat Movielens as an implicit system. This means that every movie a user watched is a positive example, and every movie they have not seen is an implicit negative example.


## Imports


Let's first get our imports out of the way.



In [0]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [0]:
import tensorflow_recommenders as tfrs

## Preparing the dataset

Let's first have a look at the data.

We use the MovieLens dataset from [Tensorflow Datasets](https://www.tensorflow.org/datasets). Specifically, loading `movie_lens/100k_ratings` yields a `tf.data.Dataset` object containing the ratings data and loading `movie_lens/100k_movies` yields a `tf.data.Dataset` object containing only the movies data.

Note that since the MovieLens dataset does not have predefined splits, all data are under `train` split.

In [0]:
# Ratings data.
ratings = tfds.load("movie_lens/100k-ratings", split="train")
# Features of all the available movies.
movies = tfds.load("movie_lens/100k-movies", split="train")

The ratings dataset returns a dictionary of movie id, user id, the assigned rating, timestamp, movie information, and user information:

In [0]:
for x in ratings.take(3).as_numpy_iterator():
  pprint.pprint(x)

{'bucketized_user_age': 45.0,
 'movie_genres': array([7]),
 'movie_id': b'357',
 'movie_title': b"One Flew Over the Cuckoo's Nest (1975)",
 'raw_user_age': 46.0,
 'timestamp': 879024327,
 'user_gender': True,
 'user_id': b'138',
 'user_occupation_label': 4,
 'user_occupation_text': b'doctor',
 'user_rating': 4.0,
 'user_zip_code': b'53211'}
{'bucketized_user_age': 25.0,
 'movie_genres': array([ 4, 14]),
 'movie_id': b'709',
 'movie_title': b'Strictly Ballroom (1992)',
 'raw_user_age': 32.0,
 'timestamp': 875654590,
 'user_gender': True,
 'user_id': b'92',
 'user_occupation_label': 5,
 'user_occupation_text': b'entertainment',
 'user_rating': 2.0,
 'user_zip_code': b'80525'}
{'bucketized_user_age': 18.0,
 'movie_genres': array([4]),
 'movie_id': b'412',
 'movie_title': b'Very Brady Sequel, A (1996)',
 'raw_user_age': 24.0,
 'timestamp': 882075110,
 'user_gender': True,
 'user_id': b'301',
 'user_occupation_label': 17,
 'user_occupation_text': b'student',
 'user_rating': 4.0,
 'user_zip_

The movies dataset contains the movie id, movie title, and data on what genres it belongs to. Note that the genres are encoded with integer labels.

In [0]:
for x in movies.take(1).as_numpy_iterator():
  pprint.pprint(x)

{'movie_genres': array([4]),
 'movie_id': b'1681',
 'movie_title': b'You So Crazy (1994)'}


In this example, we're going to focus on the ratings data. Other tutorials explore how to use the movie information data as well to improve the model quality.

We keep only the `movie_id`, `user_id`, and `movie_title` fields in the dataset.

In [0]:
ratings = ratings.map(lambda x: {
    "movie_id": x["movie_id"],
    "user_id": x["user_id"],
})
movies = movies.map(lambda x: {
    "movie_id": x["movie_id"],
    "movie_title": x["movie_title"],
})

To fit and evaluate the model, we need to split it into a training and evaluation set. In an industrial recommender system, this would most likely be done by time: the data up to time $T$ would be used to predict interactions after $T$.


In this simple example, however, let's use a random split, putting 80% of the ratings in the train set, and 20% in the test set.

In [0]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

Let's also figure out unique user ids and movie ids present in the data. 

This is important because we need to be able to map the raw values of our categorical features to embedding vectors in our models. To do that, we need a vocabulary that maps a raw feature value to an integer in a contiguous range: this allows us to look up the corresponding embeddings in our embedding tables.

In [0]:
movie_ids = movies.batch(1_000).map(lambda x: x["movie_id"])
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

unique_movie_ids = np.unique(np.concatenate(list(movie_ids)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

# We convert bytes to strings since bytes are not serializable
unique_movie_id_strings = [id.decode("utf-8") for id in unique_movie_ids]
unique_user_id_strings = [id.decode("utf-8") for id in unique_user_ids]

unique_movie_id_strings[:10]

['1', '10', '100', '1000', '1001', '1002', '1003', '1004', '1005', '1006']

## Implementing a model

This is the critical section where we choose the architecure of our model.

Because we are building a two-tower retrieval model, we can build each tower separately and then combine them in the final model.

### The query tower

Let's start with the query tower.

The first step is to decide on the dimensionality of the query and candidate representations:

In [0]:
embedding_dimension = 32

The second is to define the input features. Here, we're going to use [feature columns](https://www.tensorflow.org/tutorials/structured_data/feature_columns), to define a simple embedding layer, taking `user_id` as its only input feature. Note that we use the list of unique user ids we computed earlier as a vocabulary:

In [0]:
class UserModel(tf.keras.Model):

  def __init__(self, embedding_dimension):
    super(UserModel, self).__init__()
    # The model itself is a single embedding layer.
    # However, we could expand this to an arbitrarily complicated Keras model, as long
    # as the output is an vector `embedding_dimension` wide.
    user_features = [tf.feature_column.embedding_column(
        tf.feature_column.categorical_column_with_vocabulary_list(
            "user_id", unique_user_id_strings,
        ),
        embedding_dimension,
    )]
    self.embedding_layer = tf.keras.layers.DenseFeatures(user_features, name="user_embedding")

  def call(self, inputs):
    return self.embedding_layer(inputs)

# We initialize these models and later pass them to the full model.
user_model = UserModel(embedding_dimension)

A simple model like this corresponds exactly to a classic [matrix factorization](https://ieeexplore.ieee.org/abstract/document/4781121) approach. Defining a subclass of `tf.keras.Model` for this simple model might be an overkill. However, we could easily extend it to an arbitrarily complex model using standard Keras components, as long as we return an `embedding_dimension`-wide output at the end.

### The candidate tower

We can do the same with the candidate tower.

In [0]:
class MovieModel(tf.keras.Model):

  def __init__(self, embedding_dimension):
    super(MovieModel, self).__init__()
    movie_features = [tf.feature_column.embedding_column(
        tf.feature_column.categorical_column_with_vocabulary_list(
            "movie_id", unique_movie_id_strings,
        ),
        embedding_dimension,
    )]
    self.embedding_layer = tf.keras.layers.DenseFeatures(movie_features, name="movie_embedding")
  
  def call(self, inputs):
    return self.embedding_layer(inputs)

movie_model = MovieModel(embedding_dimension)

### Metrics

In our training data we have positive (user, movie) pairs. To figure out how good our model is, we need to compare the affinity score that the model calculates for this pair to the scores of all the other possible candidates: if the score for the positive pair is higher than for all other candidates, our model is highly accurate.

To do this, we can use the `tfrs.metrics.FactorizedTopK` metric. The metric has one required argument: the dataset of candidates that are used as implicit negatives for evaluation.

In our case, that's the `movies` dataset, converted into embeddings via our movie model:

In [0]:
metrics = tfrs.metrics.FactorizedTopK(
  candidates=movies.batch(128).map(lambda x: {"movie_id": x["movie_id"]}).map(movie_model)
)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Creating resources inside a function passed to Dataset.map() is not supported. Create each resource outside the function, and capture it inside the function to use it.
  


### Loss

The next component is the loss used to train our model. TFRS has several loss layers and tasks to make this easy.

In this instance, we'll make use of the `RetrievalTask` object: a convenience wrapper that bundles together the loss function and metric computation:

In [0]:
task = tfrs.tasks.RetrievalTask(
  corpus_metrics=metrics
)

The task itself is a Keras layer that takes the query and candidate embeddings as arguments, and returns the computed loss: we'll use that to implement the model's training loop.

### The full model

We can now put it all together into a model. TFRS exposes a base model class `tfrs.models.Model` which streamlines bulding models: all we need to do is to set up the components in the `__init__` method, and implement the `compute_loss` method, taking in the raw features and returning a loss value.

The base model will then take care of creating the appropriate training loop to fit our model.

In [0]:
class MovielensModel(tfrs.models.Model):

  def __init__(self, user_model, movie_model):
    super().__init__()
    self.movie_model: tf.keras.Model = movie_model
    self.user_model: tf.keras.Model = user_model
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model({"user_id": features["user_id"]})
    # And pick out the movie features and pass them into the movie model,
    # getting embeddings back.
    positive_movie_embeddings = self.movie_model({"movie_id": features["movie_id"]})

    # The task computes the loss and the metrics.
    return self.task(user_embeddings, positive_movie_embeddings)

## Fitting and evaluating

After defining the model, we can use standard Keras fitting and evaluation routines to fit and evaluate the model.

Let's first instantiate the model.

In [0]:
model = MovielensModel(user_model, movie_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

Then shuffle, batch, and cache the training and evaluation data.

In [0]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

Then train the  model:

In [0]:
model.fit(cached_train, epochs=3)

Epoch 1/3
10/10 [==============================] - 20s 2s/step - factorized_top_k: 0.0233 - factorized_top_k/top_1_categorical_accuracy: 1.6250e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0029 - factorized_top_k/top_10_categorical_accuracy: 0.0063 - factorized_top_k/top_50_categorical_accuracy: 0.0351 - factorized_top_k/top_100_categorical_accuracy: 0.0722 - loss: 70263.3210
Epoch 2/3
10/10 [==============================] - 20s 2s/step - factorized_top_k: 0.0977 - factorized_top_k/top_1_categorical_accuracy: 0.0011 - factorized_top_k/top_5_categorical_accuracy: 0.0145 - factorized_top_k/top_10_categorical_accuracy: 0.0321 - factorized_top_k/top_50_categorical_accuracy: 0.1599 - factorized_top_k/top_100_categorical_accuracy: 0.2809 - loss: 67848.3388
Epoch 3/3
10/10 [==============================] - 20s 2s/step - factorized_top_k: 0.1172 - factorized_top_k/top_1_categorical_accuracy: 0.0019 - factorized_top_k/top_5_categorical_accuracy: 0.0214 - factorized_top_k/top_10_catego

As the model trains, the loss is falling and a set of top-k retrieval metrics is updated. These tell us whether the true positive is in the top-k retrieved items from the entire candidate set. For example, a top-5 categorical accuracy metric of 0.2 would tell us that, on average, the true positive is in the top 5 retrieved items 20% of the time.

Note that, in this example, we evaluate the metrics during training as well as evaluation. Because this can be quite slow with large candidate sets, it may be prudent to turn metric calculation off in training, and only run it in evaluation.

Finally, we can evaluate our model on the test set:

In [0]:
model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 4s 708ms/step - factorized_top_k: 0.0644 - factorized_top_k/top_1_categorical_accuracy: 6.5000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0060 - factorized_top_k/top_10_categorical_accuracy: 0.0151 - factorized_top_k/top_50_categorical_accuracy: 0.0985 - factorized_top_k/top_100_categorical_accuracy: 0.2016 - loss: 31400.3115


{'factorized_top_k': array([0.00065, 0.006  , 0.0151 , 0.0985 , 0.2016 ], dtype=float32),
 'factorized_top_k/top_100_categorical_accuracy': 0.20160000026226044,
 'factorized_top_k/top_10_categorical_accuracy': 0.01510000042617321,
 'factorized_top_k/top_1_categorical_accuracy': 0.0006500000017695129,
 'factorized_top_k/top_50_categorical_accuracy': 0.09849999845027924,
 'factorized_top_k/top_5_categorical_accuracy': 0.006000000052154064,
 'loss': 28554.8203125}

Test set performance is much worse than training performance. This is due to two factors:

1. Our model is likely to perform better on the data that it has seen, simply because it can memorize it. This overfitting phenomenon is especially strong when models have many parameters. It can be mediated by model regularization and use of user and movie features that help the model generalize better to unseen data.
2. The model is re-recommending some of users' already watched movies. These known-positive watches can crowd out test movies out of top K recommendations.

The second phenomenon can be tackled by excluding previously seen movies from test recommendations.

In [0]:
tfrs.examples.movielens.evaluate(
    user_model=model.user_model,
    movie_model=model.movie_model,
    test=test,
    movies=movies,
    train=train,
    k=10,
)

{'precision_at_k': 0.09097664543524417, 'recall_at_k': 0.06814190082266666}

These values are higher than if we did not exclude the training set watches:

In [0]:
tfrs.examples.movielens.evaluate(
    user_model=model.user_model,
    movie_model=model.movie_model,
    test=test,
    movies=movies,
    train=None,
    k=10,
)

{'precision_at_k': 0.03343949044585987, 'recall_at_k': 0.02842858616814186}

Of course, accuracy on the training set is still much higher:

In [0]:
tfrs.examples.movielens.evaluate(
    user_model=model.user_model,
    movie_model=model.movie_model,
    test=train,
    movies=movies,
    k=10,
)

{'precision_at_k': 0.6022269353128314, 'recall_at_k': 0.12978213265599212}

## Making predictions

Now that we have a model, we would like to be able to make predictions. We can use the `DatasetIndexedTopK` layer to do this.

In [0]:
top_k = tfrs.layers.corpus.DatasetIndexedTopK(
    # We transform the movies dataset into pairs of (movie title, movie embedding)
    # to allow us to retrieve most highly scored titles given embedding.
    # We use the `cache` transformation to make sure we don't recompute
    # movie embeddings every time we score a query.
    candidates=movies.batch(4096).map(lambda x: (
        x["movie_title"],
        model.movie_model({"movie_id": x["movie_id"]})
    )).cache()
)

Now that we have the candidate layer, all that remains is to get some user embeddings and run the top k queries:

In [0]:
for user_id in ("10", "123", "557"):
  _, top_titles = top_k(model.user_model({"user_id": np.array([user_id])}))
  print(f"Top titles for user {user_id}: {top_titles}")

Top titles for user 10: [[b'Blue Angel, The (Blaue Engel, Der) (1930)' b'Big Sleep, The (1946)'
  b'Cat on a Hot Tin Roof (1958)' b"Singin' in the Rain (1952)"
  b'Apartment, The (1960)' b'To Catch a Thief (1955)'
  b'Treasure of the Sierra Madre, The (1948)'
  b'Wonderful, Horrible Life of Leni Riefenstahl, The (1993)'
  b'All About Eve (1950)' b'M (1931)']]
Top titles for user 123: [[b'Annie Hall (1977)' b"Antonia's Line (1995)" b'Secrets & Lies (1996)'
  b'Blue Angel, The (Blaue Engel, Der) (1930)' b'Roman Holiday (1953)'
  b'Treasure of the Sierra Madre, The (1948)'
  b'Philadelphia Story, The (1940)' b'Penny Serenade (1941)'
  b'It Happened One Night (1934)' b'Ruby in Paradise (1993)']]
Top titles for user 557: [[b'Welcome To Sarajevo (1997)' b'Love Jones (1997)'
  b'House of Yes, The (1997)' b"Eve's Bayou (1997)"
  b'In the Company of Men (1997)' b'Hoodlum (1997)' b'Money Talks (1997)'
  b"She's So Lovely (1997)" b'Jackie Brown (1997)'
  b'Ice Storm, The (1997)']]


## Model serving

After the model is trained, we need a way to deploy it.

In a two-tower retrieval model, serving has two components:

- a serving query model, taking in features of the query and transforming them into a query embedding, and
- a serving candidate model. This most often takes the form of an approximate nearest neighbours (ANN) index which allows fast approximate lookup of candidates in response to a query produced by the query model.

### Exporting a query model to serving

Exporting the query model is easy: we can either serialize the Keras model directly, or export it to a `SavedModel` format to make it possible to serve using [TensorFlow Serving](https://www.tensorflow.org/tfx/guide/serving).

To export to a `SavedModel` format, we can do the following:

In [0]:
# Export the query model.
with tempfile.TemporaryDirectory() as tmp:
  path = os.path.join(tmp, "query_model")
  tf.saved_model.save(model.user_model, path)
  loaded = tf.saved_model.load(path)
  infer = loaded.signatures["serving_default"]
  query_embedding = infer(user_id=tf.constant(["10"], dtype=tf.string))["output_1"]

  print(f"Query embedding: {query_embedding[0, :3]}")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


INFO:tensorflow:Assets written to: /tmp/tmpgwrqanq9/query_model/assets


INFO:tensorflow:Assets written to: /tmp/tmpgwrqanq9/query_model/assets


Query embedding: [-0.01021871  0.4163845  -0.7192347 ]


### Building a candidate ANN index

Exporting candidate representations is more involved. Firstly, we want to pre-compute them to make sure serving is fast; this is especially important if the candidate model is computationally intensive (for example, if it has many or wide layers; or uses complex representations for text or images). Secondly, we would like to take the precomputed representations and use them to construct a fast approximate retrieval index.


We can use [Annoy](https://github.com/spotify/annoy) to build such an index.

We first install annoy in case it is not available.

In [0]:
!pip3 install annoy

We now instantiate the index object.

In [0]:
from annoy import AnnoyIndex

index = AnnoyIndex(embedding_dimension, "dot")

Then take the candidate dataset and transform its raw features into embeddings using the movie model:

In [0]:
movie_embeddings = (
    movies.batch(128).map(lambda x: (
        tf.strings.to_number(x["movie_id"], out_type=tf.int32),
        model.movie_model({"movie_id": x["movie_id"]}),
    ))
)

And then index the movie_id, movie embedding pairs into our Annoy index:

In [0]:
movie_id_to_title = dict(movies.map(
    lambda x: (tf.strings.to_number(x["movie_id"], out_type=tf.int32), x["movie_title"])
).as_numpy_iterator())

# We unbatch the dataset because Annoy accepts only scalar (id, embedding) pairs.
for movie_id, movie_embedding in movie_embeddings.unbatch().as_numpy_iterator():
  index.add_item(movie_id, movie_embedding)

# Build a 10-tree ANN index.
index.build(10)

True

We can then retrieve nearest neighbours:

In [0]:
for row in test.batch(1).take(10):
  query_embedding = model.user_model(
      {"user_id": row["user_id"]}
  )[0]
  candidates = index.get_nns_by_vector(query_embedding, 3)
  print(f"Candidates: {[movie_id_to_title[x] for x in candidates]}.")


Candidates: [b'Hard Target (1993)', b'Rising Sun (1993)', b'Dangerous Minds (1995)'].
Candidates: [b'Air Bud (1997)', b"McHale's Navy (1997)", b'Jungle2Jungle (1997)'].
Candidates: [b'Tom and Huck (1995)', b'Heavyweights (1994)', b'Another Stakeout (1993)'].
Candidates: [b'Affair to Remember, An (1957)', b'French Kiss (1995)', b'American President, The (1995)'].
Candidates: [b'Raiders of the Lost Ark (1981)', b'Terminator 2: Judgment Day (1991)', b'Forbidden Planet (1956)'].
Candidates: [b'City of Lost Children, The (1995)', b'Breaking the Waves (1996)', b'Godfather, The (1972)'].
Candidates: [b'Garden of Finzi-Contini, The (Giardino dei Finzi-Contini, Il) (1970)', b'World of Apu, The (Apur Sansar) (1959)', b'Belle de jour (1967)'].
Candidates: [b'Affair to Remember, An (1957)', b'French Kiss (1995)', b'American President, The (1995)'].
Candidates: [b'Substitute, The (1996)', b'Quest, The (1996)', b'Sudden Death (1995)'].
Candidates: [b'Mortal Kombat: Annihilation (1997)', b'Mortal Kom

## Next steps

This concludes the retrieval tutorial.

To expand on what is presented here, have a look at:

1. Learning multi-task models: jointly optimizing for ratings and clicks.
2. Using movie metadata: building a more complex movie model to alleviate cold-start.